In [2]:
import pymysql.cursors
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

import json
import time

import numpy as np

from bs4 import BeautifulSoup

from pydantic import BaseModel
from typing import List, Optional

from selenium import webdriver

from bs4 import BeautifulSoup as BS
import bs4
import soupsieve

import pymysql
from typing import List, Dict, Optional
from pymysql.cursors import DictCursor

import warnings
warnings.filterwarnings('ignore')

## BIG 5 L TEAMS' WHOSCORED FIXTURES URLS

In [3]:
df_urls = pd.read_excel('C:/Users/Personnel/Ligue 1 Project/ws_url_teams.xlsx')
df_urls

,url,league,id,team
0,https://fr.whoscored.com/Teams/304/Fixtures/Fr...,France-Ligue-1,304,Paris-Saint-Germain
1,https://fr.whoscored.com/Teams/309/Fixtures/Fr...,France-Ligue-1,309,Lens
2,https://fr.whoscored.com/Teams/613/Fixtures/Fr...,France-Ligue-1,613,Nice
3,https://fr.whoscored.com/Teams/248/Fixtures/Fr...,France-Ligue-1,248,Monaco
4,https://fr.whoscored.com/Teams/607/Fixtures/Fr...,France-Ligue-1,607,Lille
...,...,...,...,...
91,https://www.whoscored.com/Teams/189/Fixtures/A...,Angleterre-Premier-League,189,Brentford
92,https://www.whoscored.com/Teams/170/Fixtures/A...,Angleterre-Premier-League,170,Fulham
93,https://www.whoscored.com/Teams/184/Fixtures/A...,Angleterre-Premier-League,184,Burnley
94,https://www.whoscored.com/Teams/163/Fixtures/A...,Angleterre-Premier-League,163,Sheffield-United


## EXTRACTING ALL THE GAMES' LINKS 

In [4]:
# Define the leagues you're interested in
leagues = ['Allemagne-Bundesliga', 'England-Premier-League', 'Espagne-LaLiga', 'France-Ligue-1', 'Italie-Serie-A']

# Initialize an empty set to store unique URLs (sets inherently avoid duplicates)
all_urls = set()

# Initialize the WebDriver
driver = webdriver.Chrome()  # or webdriver.Firefox(), etc., depending on your browser and setup

# Iterate through each row in the DataFrame
for index, row in df_urls.iterrows():
    # Extract the URL and league from the 'url' column
    team_fixtures_url = row['url']
    
    # Use the WebDriver to visit the page
    driver.get(team_fixtures_url)
    
    # Add a delay to avoid overloading the server and to ensure the page loads
    time.sleep(2)
    
    # Use BeautifulSoup to parse the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract additional URLs from the page
    # Filter out the links that have '/Live/' and belong to the specified leagues
    additional_urls = []
    for league in leagues:
        additional_urls.extend(soup.select(f'a[href*="/Live/"][href*="{league.replace(" ", "-")}"]'))
    
    # Process each found URL
    for link in additional_urls:
        href = link.get('href')
        if href:  # Make sure href is not None
            full_url = f'https://www.whoscored.com{href}'
            all_urls.add(full_url)  # Set ensures only unique URLs are added

# Convert the set back to a list if needed
unique_urls_list = list(all_urls)

# Print or use the list of all unique URLs
for url in unique_urls_list:
    print(url)

# Make sure to close the WebDriver after the scraping job is done
driver.quit()


https://www.whoscored.com/Matches/1741171/Live/France-Ligue-1-2023-2024-Metz-Strasbourg
https://www.whoscored.com/Matches/1746096/Live/Italie-Serie-A-2023-2024-Napoli-Udinese
https://www.whoscored.com/Matches/1729252/Live/England-Premier-League-2023-2024-Manchester-City-Newcastle
https://www.whoscored.com/Matches/1734711/Live/Espagne-LaLiga-2023-2024-Deportivo-Alaves-Athletic-Club
https://www.whoscored.com/Matches/1746233/Live/Italie-Serie-A-2023-2024-Frosinone-Cagliari
https://www.whoscored.com/Matches/1746240/Live/Italie-Serie-A-2023-2024-Atalanta-Udinese
https://www.whoscored.com/Matches/1746226/Live/Italie-Serie-A-2023-2024-AC-Milan-Sassuolo
https://www.whoscored.com/Matches/1734659/Live/Espagne-LaLiga-2023-2024-Villarreal-Athletic-Club
https://www.whoscored.com/Matches/1746048/Live/Italie-Serie-A-2023-2024-Udinese-Juventus
https://www.whoscored.com/Matches/1734905/Live/Espagne-LaLiga-2023-2024-Cadiz-Valencia
https://www.whoscored.com/Matches/1729491/Live/England-Premier-League-202

In [6]:
print(len(all_urls))

1093


## EXTRACTING MATCH_INFO
### MATCH_ID, LEAGUE, TEAMS...

In [7]:
def extract_match_info(url):
    # Extract match_id after 'Matches/'
    match_id_start = url.find('Matches/') + len('Matches/')
    match_id_end = url.find('/Live/')
    match_id = url[match_id_start:match_id_end]
    
    # Extract league after 'Live/'
    league_start = url.find('/Live/') + len('/Live/')
    league_end = url.find('-2023-2024')
    league = url[league_start:league_end]
    
    # Extract teams after '2024-'
    teams_start = url.find('2024-') + len('2024-')
    teams = url[teams_start:]
    
    return match_id, league, teams


connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db',
                             cursorclass=pymysql.cursors.DictCursor)

for url in unique_urls_list:
    match_id, league, teams = extract_match_info(url)
    
    # Insert the extracted data into the MySQL table
    with connection.cursor() as cursor:
        insert_query = "INSERT IGNORE INTO matchs (match_id, league, teams, match_link) VALUES (%s, %s, %s, %s)"
        data = (match_id, league, teams, url)
        cursor.execute(insert_query, data)
        connection.commit()

# Close the connection
connection.close()


## FORMATTING THE MATCH_EVENTS & PLAYERS DATA

In [8]:
class MatchEvent(BaseModel):
    id: int
    match_id: int
    event_id: int
    minute: int
    second: Optional[float] = None
    team_id: int
    player_id: int
    x: float
    y: float
    end_x: Optional[float] = None
    end_y: Optional[float] = None
    qualifiers: List[dict]
    is_touch: bool
    blocked_x: Optional[float] = None
    blocked_y: Optional[float] = None
    goal_mouth_z: Optional[float] = None
    goal_mouth_y: Optional[float] = None
    is_shot: bool
    card_type: bool
    is_goal: bool
    type_display_name: str
    outcome_type_display_name: str
    period_display_name: str

In [9]:
class Player(BaseModel):
    player_id: int
    shirt_no: int
    name: str
    age: int
    position: str
    team_id: int

## VERIFY IF A GAME IS SCRAPED OR NOT YET

def is_match_id_scraped(match_id, connection):
    with connection.cursor() as cursor:
        query = "SELECT EXISTS(SELECT 1 FROM match_events WHERE match_id = %s) AS is_scraped"
        cursor.execute(query, (match_id,))
        result = cursor.fetchone()
        return bool(result['is_scraped'])  # Converts 1/0 to True/False

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    match_id_to_check = '1729199'  # Replace with the actual match_id
    is_scraped = is_match_id_scraped(match_id_to_check, connection)
    if is_scraped:
        print(f"Match with ID {match_id_to_check} has already been scraped.")
    else:
        print(f"Match with ID {match_id_to_check} has not been scraped yet.")
finally:
    connection.close()


## SCRAPPING MATCH_EVENTS DATA

In [11]:
def scrape_match_events(whoscored_url, driver, connection):
    
    match_id = int(whoscored_url.split('/Matches/')[-1].split('/Live/')[0])
    
    if is_match_id_scraped(match_id, connection):
        print(f"Match with ID {match_id} has already been scraped.")
        return  # Skip this match
    
    driver.get(whoscored_url)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    element = soup.select_one('script:-soup-contains("matchCentreData")')
    
    matchdict = json.loads(element.text.split("matchCentreData: ")[1].split(',\n')[0])
    
    match_events = matchdict['events']
    
    df = pd.DataFrame(match_events)
    
    df.dropna(subset='playerId', inplace=True)
    
    df = df.where(pd.notnull(df), None)
    
    df['match_id'] = match_id
    
    if 'match_id' not in df.columns:
        raise KeyError("Column 'match_id' missing from DataFrame")
    
    df = df.rename(
    {
        'eventId': 'event_id',
        'expandedMinute': 'expanded_minute',
        'outcomeType': 'outcome_type',
        'isTouch': 'is_touch',
        'playerId': 'player_id',
        'teamId': 'team_id',
        'endX': 'end_x',
        'endY': 'end_y',
        'blockedX': 'blocked_x',
        'blockedY': 'blocked_y',
        'goalMouthZ': 'goal_mouth_z',
        'goalMouthY': 'goal_mouth_y',
        'isShot': 'is_shot',
        'cardType': 'card_type',
        'isGoal': 'is_goal'
    },
        axis=1
    )
    
    df['period_display_name'] = df['period'].apply(lambda x: x['displayName'])
    df['type_display_name'] = df['type'].apply(lambda x: x['displayName'])
    df['outcome_type_display_name'] = df['outcome_type'].apply(lambda x: x['displayName'])
    
    df.drop(columns=["period", "type", "outcome_type"], inplace=True)
    
    if 'is_goal' not in df.columns:
        df['is_goal'] = False
        
    if 'is_card' not in df.columns:
        df['is_card'] = False
        df['card_type'] = False
        
    df = df[~(df['type_display_name'] == "OffsideGiven")]
    
    df = df[[
        'id', 'event_id', 'minute', 'second', 'team_id', 'player_id', 'x', 'y', 'end_x', 'end_y',
        'qualifiers', 'is_touch', 'blocked_x', 'blocked_y', 'goal_mouth_z', 'goal_mouth_y', 'is_shot',
        'card_type', 'is_goal', 'type_display_name', 'outcome_type_display_name',
        'period_display_name','match_id'
    ]]
    
    df[['id', 'event_id', 'minute', 'team_id', 'player_id','match_id']] = df[['id', 'event_id', 'minute', 'team_id', 'player_id','match_id']].astype(np.int64)
    df[['second', 'x', 'y', 'end_x', 'end_y']] = df[['second', 'x', 'y', 'end_x', 'end_y']].astype(float)
    df[['is_shot', 'is_goal', 'card_type']] = df[['is_shot', 'is_goal', 'card_type']].astype(bool)
    
    df['is_goal'] = df['is_goal'].fillna(False)
    df['is_shot'] = df['is_shot'].fillna(False)
    
    for column in df.columns:
        if df[column].dtype == np.float64 or df[column].dtype == np.float32:
            df[column] = np.where(
                np.isnan(df[column]),
                None,
                df[column]
            )
                
    
    team_info = []
    team_info.append({
        'team_id': matchdict['home']['teamId'],
        'name': matchdict['home']['name'],
        'country_name': matchdict['home']['countryName'],
        'manager_name': matchdict['home']['managerName'],
        'players': matchdict['home']['players'],
    })

    team_info.append({
        'team_id': matchdict['away']['teamId'],
        'name': matchdict['away']['name'],
        'country_name': matchdict['away']['countryName'],
        'manager_name': matchdict['away']['managerName'],
        'players': matchdict['away']['players'],
    })
    
    #print(df.head())
    
    insert_players(team_info, connection)
    insert_match_events(df, connection)
    
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db',
                             cursorclass=pymysql.cursors.DictCursor)    


## INSERTING THE DATA INTO MYSQL

In [12]:
def insert_players(team_info, connection):
    insert_query = """
    INSERT INTO players (player_id, name, shirt_no, age, position, team_id)
    VALUES (%(player_id)s, %(name)s, %(shirt_no)s, %(age)s, %(position)s, %(team_id)s)
    ON DUPLICATE KEY UPDATE name = VALUES(name), shirt_no = VALUES(shirt_no), age = VALUES(age), position = VALUES(position);
    """
    with connection.cursor() as cursor:
        for team in team_info:
            for player in team['players']:
                # Create a dictionary for the player
                player_data = {
                    'player_id': player['playerId'],
                    'name': player['name'],
                    'shirt_no': player.get('shirtNo', None),  # Using .get() to handle missing data
                    'age': player.get('age', None),
                    'position': player.get('position', None),
                    'team_id': team['team_id']
                }
                cursor.execute(insert_query, player_data)
        connection.commit()


def insert_match_events(df, connection):
    with connection.cursor() as cursor:
        for _, row in df.iterrows():
            insert_query = """
                INSERT INTO match_events 
                (id, match_id, event_id, minute, second, team_id, player_id, x, y, end_x, end_y, qualifiers, is_touch, blocked_x, blocked_y, goal_mouth_z, goal_mouth_y, is_shot, card_type, is_goal, type_display_name, outcome_type_display_name, period_display_name) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                minute = VALUES(minute), second = VALUES(second), x = VALUES(x), y = VALUES(y), end_x = VALUES(end_x), end_y = VALUES(end_y), is_shot = VALUES(is_shot), is_goal = VALUES(is_goal)
            """
            # Prepare your data tuple from the row here
            data = (row['id'], row['match_id'], row['event_id'], row['minute'], row['second'], row['team_id'], row['player_id'], row['x'], row['y'], row['end_x'], row['end_y'], json.dumps(row['qualifiers']), row['is_touch'], row['blocked_x'], row['blocked_y'], row['goal_mouth_z'], row['goal_mouth_y'], row['is_shot'], row['card_type'], row['is_goal'], row['type_display_name'], row['outcome_type_display_name'], row['period_display_name'])
            
            cursor.execute(insert_query, data)
        connection.commit()


## IDENTIFY UNSCRAPED GAMES

In [13]:
def create_connection():
    return pymysql.connect(host='localhost',
                           user='root',
                           password='Darwin@nunez9',
                           database='2023_24_db',
                           cursorclass=pymysql.cursors.DictCursor)

def get_unscraped_urls(all_urls):
    new_urls = []
    connection = create_connection()
    try:
        with connection.cursor() as cursor:
            for url in all_urls:
                match_id = url.split('/Matches/')[-1].split('/Live/')[0]
                query = "SELECT EXISTS(SELECT 1 FROM match_events WHERE match_id = %s) AS is_scraped"
                cursor.execute(query, (match_id,))
                result = cursor.fetchone()
                if not bool(result['is_scraped']):  # If not scraped, add to new_urls
                    new_urls.append(url)
    except pymysql.MySQLError as e:
        print(f"An error occurred: {e}")
    finally:
        connection.close()
    return new_urls

# Assuming all_urls is your list of URLs
new_urls = get_unscraped_urls(all_urls)
len(new_urls)

0

## SCRAPE THE UNSCRAPED GAMES

In [14]:
import warnings
warnings.filterwarnings('ignore')

driver = webdriver.Chrome()

for i,url in enumerate(new_urls,start=1):
    print(f"Processing {i}/{len(new_urls)}: {url}")
    try:
        scrape_match_events(whoscored_url=url, driver=driver, connection=connection)
    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
    finally:
        # Wait a bit before scraping the next page to avoid being too aggressive
        time.sleep(2)

## INSERTING THE TEAM INFOS INTO MYSQL DB

pip install sqlalchemy

## SCRAPPING THE PLAYERS_INFO
### Minutes played, Positions, Rating...

In [24]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

def scrape_and_insert_players_info(team_url, driver, connection):
    # Initialize the Chrome WebDriver
    driver = webdriver.Chrome()
    driver.get(team_url)

    # Wait a fixed amount of time for the page to load
    time.sleep(5)  # Adjusted from 30 seconds for brevity

    data = []  # List to hold all rows of data

    try:
        # Once the wait is over, proceed with scraping
        element_html = driver.find_element(By.ID, "team-squad-stats").get_attribute('outerHTML')
        soup = BeautifulSoup(element_html, 'html.parser')

        rows = soup.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            player_link = row.find('a', class_='player-link')  # Find the player link in the row
            if cells and player_link:  # Making sure it's not a header row and player link exists
                player_id = player_link.get('href').split('/')[2]  # Extracting player ID from the href
                row_data = [cell.get_text(strip=True) for cell in cells]
                row_data.append(player_id)  # Append player ID to the row data
                data.append(row_data)
    finally:
        driver.quit()

    # Adjust the column names to include 'Player ID'
    column_names = ['index', 'Name', 'Height', 'Weight', 'Appearances', 'Minutes', 'Goals', 
                    'Assists', 'Yellow Cards', 'Red Cards', 'Shots/Game', 'Passing Accuracy', 
                    'Aerials Won/Game', 'Man of the Match', 'Rating', 'Player_id'] 

    # Convert the list of data into a DataFrame
    df = pd.DataFrame(data, columns=column_names)

    # Extract 'Age' first by finding two consecutive digits
    df['Age'] = df['Name'].str.extract(r'(\d{2})')

    # Extract 'Name_clean' by getting all characters before 'Age'
    # The regular expression uses a negative look ahead to ensure we stop before the 'Age' digits
    df['Name_clean'] = df['Name'].str.extract(r'^(.*?)(?=\d{2})')

    # Extract 'Positions' by getting all characters after 'Age'
    # We use a positive lookbehind to find the position after the 'Age' digits
    df['Positions'] = df['Name'].str.extract(r'(?<=\d{2})(.*)')
    df['Positions'] = df['Positions'].str.lstrip(', ')

    # Now we can clean up the 'Name_clean' to remove any trailing whitespace or commas
    df['Name_clean'] = df['Name_clean'].str.strip(', ')

    # Convert 'Age' to integer, filling NaNs with a default value or zero
    df['Age'] = df['Age'].fillna(0).astype(int)

    # Fill NaN values in 'Positions' with an empty string
    df['Positions'] = df['Positions'].fillna('')

    # Rename 'Name_clean' to 'Name' and drop the old 'Name' column
    df.drop(columns=['Name'], inplace=True)
    df.rename(columns={'Name_clean': 'Name'}, inplace=True)

    # Assuming the rest of the columns are correctly named and don't need changes
    # Reorder the DataFrame to match the desired structure
    column_order = ['Name', 'Player_id','Age', 'Positions', 'Height', 'Weight', 'Minutes','Appearances', 'Goals', 'Assists', 
                    'Yellow Cards', 'Red Cards', 'Shots/Game', 'Passing Accuracy', 'Aerials Won/Game', 
                    'Man of the Match', 'Rating']
    df = df[column_order]
    
        # Split the 'Appearances' column at the parentheses and keep the first part
    df['Appearances'] = df['Appearances'].str.split('(').str[0]

    # Convert the 'Appearances' column to an integer type
    # Convert columns to numeric types where appropriate
    numeric_cols = ['Height', 'Weight', 'Minutes', 'Appearances', 'Goals', 'Assists', 
                    'Yellow Cards', 'Red Cards', 'Shots/Game', 'Passing Accuracy', 
                    'Aerials Won/Game', 'Man of the Match', 'Rating']

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df['Shots/Game'] = df['Shots/Game'].fillna(0)
    df['Aerials Won/Game'] = df['Aerials Won/Game'].fillna(0)
    df['Passing Accuracy'] = df['Passing Accuracy'].fillna(0)

    df['Man of the Match'] = df['Man of the Match'].fillna(0).astype(int)

    int_cols = ['Appearances', 'Goals', 'Assists', 'Yellow Cards', 'Red Cards']

    for col in int_cols:
        df[col] = df[col].fillna(0).astype(int)

    # Assuming you add your data cleaning/transformation logic here

    team_id = int(team_url.split('/Teams/')[-1].split('/Show/')[0])
    
    df['team_id'] = team_id

    #return df
    insert_players_info_to_mysql(df, connection)
    
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db',
                             cursorclass=pymysql.cursors.DictCursor)    
    
#team_url = 'https://fr.whoscored.com/Teams/26/Show/Angleterre-Liverpool'
#scrape_and_insert_players_info(team_url)

In [25]:
import pymysql

def insert_players_info_to_mysql(df, connection):
    with connection.cursor() as cursor:
        for _, row in df.iterrows():
            insert_query = """
                INSERT INTO players_info 
                (player_id, name, positions, height, weight, appearances, minutes, goals, assists, yellow_cards, red_cards, shots_per_game, passing_accuracy, aerials_won_per_game, man_of_the_match, rating, team_id) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                name=VALUES(name), positions=VALUES(positions), height=VALUES(height), weight=VALUES(weight), appearances=VALUES(appearances), minutes=VALUES(minutes), goals=VALUES(goals), assists=VALUES(assists), yellow_cards=VALUES(yellow_cards), red_cards=VALUES(red_cards), shots_per_game=VALUES(shots_per_game), passing_accuracy=VALUES(passing_accuracy), aerials_won_per_game=VALUES(aerials_won_per_game), man_of_the_match=VALUES(man_of_the_match), rating=VALUES(rating), team_id=VALUES(team_id);
            """
            # Prepare your data tuple from the row here
            data = (row['Player_id'], row['Name'], row['Positions'], row['Height'], row['Weight'], row['Appearances'], row['Minutes'], row['Goals'], row['Assists'], row['Yellow Cards'], row['Red Cards'], row['Shots/Game'], row['Passing Accuracy'], row['Aerials Won/Game'], row['Man of the Match'], row['Rating'], row['team_id'])
            
            cursor.execute(insert_query, data)
        connection.commit()

In [26]:
df_urls['show_url'] = df_urls['url'].apply(lambda x: x.replace('Fixtures', 'Show'))

for index, row in df_urls.iterrows():
    team_url = row['show_url']  # Get the team URL from the current row
    print(f"Processing: {team_url}")  # Optional: print the URL being processed for tracking progress
    try:
        scrape_and_insert_players_info(team_url, driver, connection)  # Call your function with the current team URL
    except Exception as e:
        print(f"Failed to process {team_url}: {e}")  # Handle any errors during processing

connection.close()

Processing: https://fr.whoscored.com/Teams/304/Show/France-Paris-Saint-Germain
Processing: https://fr.whoscored.com/Teams/309/Show/France-Lens
Processing: https://fr.whoscored.com/Teams/613/Show/France-Nice
Processing: https://fr.whoscored.com/Teams/248/Show/France-Monaco
Processing: https://fr.whoscored.com/Teams/607/Show/France-Lille
Processing: https://fr.whoscored.com/Teams/2332/Show/France-Brest
Processing: https://fr.whoscored.com/Teams/249/Show/France-Marseille
Processing: https://fr.whoscored.com/Teams/950/Show/France-Reims
Processing: https://fr.whoscored.com/Teams/302/Show/France-Nantes
Processing: https://fr.whoscored.com/Teams/148/Show/France-Strasbourg
Processing: https://fr.whoscored.com/Teams/217/Show/France-Le-Havre
Processing: https://fr.whoscored.com/Teams/314/Show/France-Metz
Processing: https://fr.whoscored.com/Teams/313/Show/France-Rennes
Processing: https://fr.whoscored.com/Teams/311/Show/France-Montpellier
Processing: https://fr.whoscored.com/Teams/246/Show/Franc

## CALL THE TABLES FROM SQL

In [57]:
# Establish a connection to the MySQL database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db')
                             #cursorclass=pymysql.cursors.DictCursor)    

# Define your SQL query
query = "SELECT * FROM match_events"

# Use pandas to load the query result into a DataFrame
df_events = pd.read_sql(query, connection)

# Close the connection
connection.close()

# Now you can work with the DataFrame 'df'
df_events

,id,match_id,event_id,minute,second,team_id,player_id,x,y,end_x,...,blocked_x,blocked_y,goal_mouth_z,goal_mouth_y,is_shot,card_type,is_goal,type_display_name,outcome_type_display_name,period_display_name
0,2575371175,1734621,4,0,0.0,64,422960,50.1,50.0,32.1,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
1,2575371189,1734621,5,0,2.0,64,80758,29.0,63.8,76.0,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Unsuccessful,FirstHalf
2,2575371241,1734621,4,0,17.0,1799,338365,24.2,0.0,34.7,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Successful,FirstHalf
3,2575371247,1734621,5,0,18.0,1799,402214,34.7,5.3,38.0,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Unsuccessful,FirstHalf
4,2575371259,1734621,7,0,19.0,64,236575,60.3,83.7,70.2,...,NaN,NaN,NaN,NaN,0,0,0,Pass,Unsuccessful,FirstHalf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695038,2645915997,1729472,1241,95,36.0,15,395692,94.0,4.6,NaN,...,NaN,NaN,NaN,NaN,0,0,0,TakeOn,Successful,SecondHalf
1695039,2645916003,1729472,650,95,36.0,162,409440,6.0,95.4,NaN,...,NaN,NaN,NaN,NaN,0,0,0,Challenge,Unsuccessful,SecondHalf
1695040,2645918771,1746266,973,86,26.0,87,439909,45.4,5.1,NaN,...,NaN,NaN,NaN,NaN,0,0,0,Dispossessed,Successful,SecondHalf
1695041,2645935047,1734814,1124,83,0.0,1799,381644,64.1,61.9,NaN,...,NaN,NaN,NaN,NaN,0,0,0,BallTouch,Successful,SecondHalf


In [58]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db')
                             #cursorclass=pymysql.cursors.DictCursor)    

# Define your SQL query
query = "SELECT * FROM players"

# Use pandas to load the query result into a DataFrame
df_players = pd.read_sql(query, connection)

# Close the connection
connection.close()

# Now you can work with the DataFrame 'df'
df_players

,player_id,shirt_no,name,age,position,team_id
0,2987,1,Pepe Reina,41,Sub,839
1,4511,6,James Milner,38,MC,211
2,6227,14,Dani Rodríguez,35,MC,51
3,6683,4,Dante,40,DC,613
4,8105,31,Andy Lonergan,40,Sub,31
...,...,...,...,...,...,...
3130,516051,41,Stanis Idumbo Muzambo,18,Sub,67
3131,516354,56,Luca Napolitano,20,Sub,77
3132,516363,43,Tim Schleinitz,18,Sub,796
3133,516373,33,Alejandro Herranz,19,Sub,819


## TEST  : EXEMPLE QUERY DATA (MISSED SHOTS)

In [27]:
import pandas as pd
import pymysql

# Database connection parameters
db_connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db')

# Your SQL query
sql_query = """
SELECT 
  p.name AS player_name,
  t.team AS team_name,
  COUNT(*) AS missed_shots
FROM 
  match_events AS me
JOIN 
  players AS p ON me.player_id = p.player_id
JOIN 
  teams AS t ON p.team_id = t.team_id
WHERE 
  me.type_display_name = 'MissedShots'
GROUP BY 
  p.name, t.team
ORDER BY 
  missed_shots DESC
LIMIT 20;
"""

# Execute the query and store the result in a pandas DataFrame
df_missed_shots = pd.read_sql(sql_query, db_connection)

# Make sure to close the database connection
db_connection.close()

# Display the DataFrame
df_missed_shots

,player_name,team_name,missed_shots
0,Nikola Krstovic,Lecce,28
1,Rodrygo,Real-Madrid,27
2,Dominic Solanke,Bournemouth,27
3,Dusan Vlahovic,Juventus,25
4,Alassane Pléa,Borussia-M-Gladbach,25
5,Tim Kleindienst,FC-Heidenheim,25
6,Chris Ramos,Cadiz,25
7,Kevin Behrens,Union-Berlin,24
8,Cyle Larin,Mallorca,24
9,Loïs Openda,RB-Leipzig,24


In [31]:
# Database connection parameters
db_connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db')

# Your SQL query
sql_query = """
SELECT 
    p.name AS player_name,
    t.team AS team_name,
    SUM(CASE WHEN me.type_display_name = 'MissedShots' THEN 1 ELSE 0 END) AS missed_shots,
    SUM(CASE WHEN me.is_shot = True THEN 1 ELSE 0 END) AS total_shots,
    (SUM(CASE WHEN me.type_display_name = 'MissedShots' THEN 1 ELSE 0 END) / SUM(CASE WHEN me.is_shot = True THEN 1 ELSE 0 END)) * 100 AS percentage_missed
FROM 
    match_events AS me
JOIN 
    players AS p ON me.player_id = p.player_id
JOIN 
    teams AS t ON p.team_id = t.team_id
GROUP BY 
    p.name, t.team
HAVING 
    SUM(CASE WHEN me.is_shot = True THEN 1 ELSE 0 END) > 20
ORDER BY 
    percentage_missed DESC
"""

# Execute the query and store the result in a pandas DataFrame
df_missed_shots = pd.read_sql(sql_query, db_connection)

# Make sure to close the database connection
db_connection.close()

# Display the DataFrame
df_missed_shots

,player_name,team_name,missed_shots,total_shots,percentage_missed
0,Beto,Everton,19.0,26.0,73.0769
1,Rubén Alcaraz,Cadiz,15.0,24.0,62.5000
2,Federico Gatti,Juventus,13.0,21.0,61.9048
3,Michael Folorunsho,Verona,19.0,32.0,59.3750
4,Miguel Gutiérrez,Girona,14.0,24.0,58.3333
...,...,...,...,...,...
447,Mikel Oyarzabal,Real-Sociedad,4.0,28.0,14.2857
448,Takumi Minamino,Monaco,4.0,32.0,12.5000
449,Borja Mayoral,Getafe,5.0,44.0,11.3636
450,Chiedozie Ogbene,Luton,2.0,22.0,9.0909


In [33]:
# Database connection parameters
db_connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Darwin@nunez9', 
                             database='2023_24_db')

# Your SQL query
sql_query = """
SELECT 
    p.name AS player_name,
    t.team AS team_name,
    SUM(CASE WHEN me.type_display_name = 'MissedShots' THEN 1 ELSE 0 END) AS missed_shots,
    SUM(CASE WHEN me.is_shot = True THEN 1 ELSE 0 END) AS total_shots,
    (SUM(CASE WHEN me.type_display_name = 'MissedShots' THEN 1 ELSE 0 END) / SUM(CASE WHEN me.is_shot = True THEN 1 ELSE 0 END)) * 100 AS percentage_missed
FROM 
    match_events AS me
JOIN 
    players AS p ON me.player_id = p.player_id
JOIN 
    teams AS t ON p.team_id = t.team_id
WHERE
    t.league = 'Angleterre-Premier-League'
GROUP BY 
    p.name, t.team
HAVING 
    SUM(CASE WHEN me.is_shot = True THEN 1 ELSE 0 END) > 20
ORDER BY 
    percentage_missed DESC
"""

# Execute the query and store the result in a pandas DataFrame
df_missed_shots_pl = pd.read_sql(sql_query, db_connection)

# Make sure to close the database connection
db_connection.close()

# Display the DataFrame
df_missed_shots_pl

,player_name,team_name,missed_shots,total_shots,percentage_missed
0,Beto,Everton,19.0,26.0,73.0769
1,Kai Havertz,Arsenal,16.0,30.0,53.3333
2,Casemiro,Manchester-United,11.0,21.0,52.3810
3,Lucas Paquetá,West-Ham,11.0,21.0,52.3810
4,Jefferson Lerma,Crystal-Palace,13.0,26.0,50.0000
...,...,...,...,...,...
110,Justin Kluivert,Bournemouth,4.0,23.0,17.3913
111,João Pedro,Brighton,8.0,48.0,16.6667
112,Neal Maupay,Brentford,6.0,37.0,16.2162
113,Chiedozie Ogbene,Luton,2.0,22.0,9.0909


In [36]:
df_missed_shots_lfc = df_missed_shots_pl[df_missed_shots_pl['team_name'] == 'Liverpool']
df_missed_shots_lfc

,player_name,team_name,missed_shots,total_shots,percentage_missed
19,Virgil van Dijk,Liverpool,10.0,25.0,40.0000
22,Trent Alexander-Arnold,Liverpool,12.0,30.0,40.0000
29,Cody Gakpo,Liverpool,11.0,29.0,37.9310
52,Dominik Szoboszlai,Liverpool,13.0,41.0,31.7073
62,Darwin Núñez,Liverpool,23.0,78.0,29.4872
76,Luis Díaz,Liverpool,13.0,47.0,27.6596
85,Mohamed Salah,Liverpool,17.0,65.0,26.1538
96,Diogo Jota,Liverpool,8.0,34.0,23.5294
